All quotes below are excerpted from https://toadstyle.org/cryptopals/60.txt

---
```
60. Single-Coordinate Ladders and Insecure Twists

All our hard work is about to pay some dividends. Here's a list of
cool-kids jargon you'll be able to deploy after completing this
challenge:

* Montgomery curve
* single-coordinate ladder
* isomorphism
* birational equivalence
* quadratic twist
* trace of Frobenius

Not that you'll understand it all; you won't. But you'll at least be
able to silence crypto-dilettantes on Twitter.

Now, to the task at hand. In the last problem, we implemented ECDH
using a short Weierstrass curve form, like this:

    y^2 = x^3 + a*x + b

For a long time, this has been the most popular curve form. The NIST
P-curves standardized in the 90s look like this. It's what you'll see
first in most elliptic curve tutorials (including this one).

We can do a lot better. Meet the Montgomery curve:

    B*v^2 = u^3 + A*u^2 + u

Although it's almost as old as the Weierstrass form, it's been buried
in the literature until somewhat recently. The Montgomery curve has a
killer feature in the form of a simple and efficient algorithm to
compute scalar multiplication: the Montgomery ladder.

Here's the ladder:

    function ladder(u, k):
        u2, w2 := (1, 0)
        u3, w3 := (u, 1)
        for i in reverse(range(bitlen(p))):
            b := 1 & (k >> i)
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
            u3, w3 := ((u2*u3 - w2*w3)^2,
                       u * (u2*w3 - w2*u3)^2)
            u2, w2 := ((u2^2 - w2^2)^2,
                       4*u2*w2 * (u2^2 + A*u2*w2 + w2^2))
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
        return u2 * w2^(p-2)

You are not expected to understand this.

Go ahead and implement the ladder. Remember that all computations are in GF(233970423115425145524320034830162017933).
```

In [1]:
from dataclasses import dataclass
from functools import reduce
from datetime import datetime
from operator import mul
from random import randrange
from math import log, ceil
from pprint import pprint

from challenge_31 import do_sha256, hmac
from challenge_57 import primegen, int_to_bytes, mini_crt, crt
from challenge_59 import tonelli_shanks, NoQuadraticResidueError

In [2]:
# As of Python 3.8 we have a builtin modular inverse function!
# Instead of challenge_39.invmod(n, p), we can just use pow(n, -1, p)
# Check it out:

def test_builtin_modular_inverse():
    from challenge_39 import invmod, InvModException
    from sys import version
    assert version >= '3.8'
    print("Profiling challenge_39's invmod() vs python's builtin pow():")
    %timeit invmod(47056997434983591416443657936535857619, 233970423115425145524320034830162017933)
    %timeit pow(47056997434983591416443657936535857619, -1, 233970423115425145524320034830162017933)

    # we'll use pow() for performance - but first let's make sure its behavior matches our invmod() function

    p = 233970423115425145524320034830162017933   # same p as our curves
    for _ in range(10000):
        while True:  # find some n such that n has an inverse mod p
            n = randrange(0, p)
            try: n_inv = invmod(n, p)
            except InvModException: continue
            break
        assert n_inv == pow(n, -1, p)
    print("Looks good!")

test_builtin_modular_inverse()

Profiling challenge_39's invmod() vs python's builtin pow():
23.9 µs ± 5.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.57 µs ± 44.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Looks good!


In [3]:
@dataclass
class MontyCurve:
    # reference for formulas: https://www.hyperelliptic.org/EFD/g1p/auto-montgom.html
    
    a: int
    b: int
    p: int
    
    def __post_init__(self):
        blp = int(ceil(log(self.p, 2)))   # bitlength of p
        self._i_range = range(blp)[::-1]  # used by mul
    
    def add_pts(self, uv1, uv2):
        # special cases: either point is zero
        if uv1 == (0, 1): return uv2
        if uv2 == (0, 1): return uv1

        u1, v1 = uv1
        u2, v2 = uv2

        # special cases: points are equal or are inverses
        if u1 == u2:
            if v1 == v2:
                return self.double(uv1)
            else:
                assert v2 == -v1 % self.p
                return (0, 1)

        # general case
        a, b, p = self.a, self.b, self.p
        
        du = (u2-u1) % p
        dv = (v2-v1) % p
        dui = pow(du, -1, p)  # lol dui

        u3 = (b * pow(dv, 2, p) * pow(dui, 2, p) - a - u1 - u2) % p
        v3 = ((2*u1 + u2 + a)*dv*dui - b*pow(dv, 3, p)*pow(dui, 3, p) - v1) % p

        return (u3, v3)
    
    def sub_pts(self, uv1, uv2):  # convenience wrapper on add_pts
        u2, v2 = uv2
        return self.add_pts(uv1, (u2, -v2 % self.p))

    def double(self, uv):
        if uv == (0, 1): return uv

        a, b, p = self.a, self.b, self.p
        u, v = uv
        usq = pow(u, 2, p)
        
        u3 = (b*pow(3*usq + 2*a*u+1, 2, p) * pow(2*b*v, -2, p)-a-u-u) % p
        v3 = ((2*u+u+a)*(3*usq+2*a*u+1)*pow(2*b*v, -1, p) - b*pow(3*usq+2*a*u+1, 3, p)*pow(2*b*v, -3, p) - v) % p
        
        return (u3, v3)

    def get_v(self, u):  # returns both possible values for v
        a, b, p = self.a, self.b, self.p
        assert b == 1  # simplifies the following eqns
        rhs = (pow(u, 3, p) + a*pow(u, 2, p) + u) % p
        v_sq = (rhs * pow(b, -1, p)) % p
        v1, v2 = tonelli_shanks(rhs, p)
        return v1, v2
    
    def mul(self, u, k):
        a = self.a
        p = self.p
        u2 = 1
        w2 = 0
        u3 = u
        w3 = 1
        for i in self._i_range:
            b = 1 & (k >> i)
            u2, u3 = (u3, u2) if b else (u2, u3)  # note: branching, not arithmetic, implementation of cswap
            w2, w3 = (w3, w2) if b else (w2, w3)  # used here for performance (see profiling results below)
            u3, w3 = ((u2*u3 - w2*w3)**2 % p,
                       u * (u2*w3 - w2*u3)**2 % p)
            u2, w2 = ((u2**2 - w2**2)**2 % p,
                       4*u2*w2 * (u2**2 + a*u2*w2 + w2**2) % p)
            u2, u3 = (u3, u2) if b else (u2, u3)
            w2, w3 = (w3, w2) if b else (w2, w3)
        return (u2 * pow(w2, p-2, p)) % p
    
    def get_pt(self, k, negate=False):
        # convenience function for getting full points instead of just x-coords
        # seems like this shouldn't be necessary but hey - if it works it works
        u = self.mul(4, k)
        u_inc = self.mul(4, k+1)
        v1, v2 = self.get_v(u)
        uv1, uv2 = (u, v1), (u, v2)
        if self.add_pts(m_base, uv1)[0] == u_inc:
            return uv2 if negate else uv1
        assert self.add_pts(m_base, uv2)[0] == u_inc
        return uv1 if negate else uv2


# the following cswap function is not used but is inlined in MontyCurve.mul()
#def cswap(a, b, i):
#    return (b, a) if i else (a, b)  # absurdly, in python this is faster than the arithmetic implementation
#    #return (b*i + a*(1-i), a*i + b*(1-i))


# ==== Profiling ====
# %timeit get_extra_coeffs()
# for 3 different implementations of MontyCurve.mul:
# 2.87 s ± 66.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap function (branching)
# 2.68 s ± 64.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap inlined (branching)
# 3.14 s ± 33.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap inlined (arithmetic)

---
```
Oh yeah, the curve parameters. You might be thinking that since we're
switching to a new curve format, we also need to pick out a whole new
curve. But you'd be totally wrong! It turns out that some short
Weierstrass curves can be converted into Montgomery curves.

You can perform this conversion algebraically. But it's kind of a
pain, so here you go:

    v^2 = u^3 + 534*u^2 + u

Through cunning and foresight, I have chosen this curve specifically
to have a really simple map between Weierstrass and Montgomery
forms. Here it is:

    u = x - 178
    v = y

Which makes our base point:

    (4, 85518893674295321206118380980485522083)

Or, you know. Just 4.

Anyway, implement the ladder. Verify ladder(4, n) = 0.
```

In [4]:
curve = MontyCurve(a=534, b=1, p=233970423115425145524320034830162017933)
m_base = (4, 85518893674295321206118380980485522083)
order = 233970423115425145498902418297807005944  # copied over from 59.txt

In [5]:
def test_monty_curve():
    assert curve.mul(4, order) == 0
    assert curve.mul(4, 0) == 0
    assert m_base == curve.get_pt(1)
    assert curve.add_pts(curve.get_pt(17), curve.get_pt(100)) == curve.get_pt(117)
    assert curve.add_pts(curve.get_pt(2**20), curve.get_pt(2**25)) == curve.get_pt(2**20 + 2**25)
    assert curve.sub_pts(curve.get_pt(100), curve.get_pt(17)) == curve.get_pt(83)

    pt_1 = pt_2 = m_base
    for i in range(1, 100):
        assert curve.add_pts(pt_1, m_base) == curve.add_pts(m_base, pt_1)

        pt_1 = curve.add_pts(m_base, pt_1)
        pt_2 = curve.add_pts(pt_2, pt_2)
        
        assert pt_1[0] == curve.mul(4, i+1)
        assert pt_2[0] == curve.mul(4, 2**i)

    print("Tests passed!")


def profile_monty_curve():
    s, t = curve.get_pt(curve.mul(4, 300))
    u, v = curve.get_pt(curve.mul(4, 1337))
    print("Execution times for mul, get_v, add_pts:")
    %timeit curve.mul(4, 1337)
    %timeit curve.get_v(u)
    %timeit curve.add_pts((s, t), (u, v))


test_monty_curve()
#profile_monty_curve()

Tests passed!


---
```
Map some points back and forth between your Weierstrass and Montgomery
representations and verify them.
```

In [6]:
from challenge_59 import Curve as WeierCurve
w_curve = WeierCurve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)
w_base = (182, 85518893674295321206118380980485522083)

In [7]:
def to_monty(pt):
    if pt is w_curve.zero:
        return 0
    x = pt[0]
    return x - 178


def to_weier(u):
    if u == 0:
        return w_curve.zero
    p, a = curve.p, curve.a
    v1, v2 = tonelli_shanks(
        (pow(u, 3, p) + a*pow(u, 2, p) + u),
        p
    )
    return (u + 178, v1), (u + 178, v2)  # two possibilities


def test_monty_weier_conversion():
    assert to_monty(w_base) == 4
    
    for _ in range(50):
        i = randrange(0, 10000)
        w = w_curve.mul(w_base, i)
        m = curve.mul(4, i)
        
        assert to_monty(w) == m
        assert w in to_weier(m)
        
    print("All good!", flush=True)

test_monty_weier_conversion()

All good!


---
```
One nice thing about the Montgomery ladder is its lack of special
cases. Specifically, no special handling of: P1 = O; P2 = O; P1 = P2;
or P1 = -P2. Contrast that with our Weierstrass addition function and
its battalion of ifs.

And there's a security benefit, too: by ignoring the v coordinate, we
take away a lot of leeway from the attacker. Recall that the ability
to choose arbitrary (x, y) pairs let them cherry-pick points from any
curve they can think of. The single-coordinate ladder robs the
attacker of that freedom.

But hang on a tick! Give this a whirl:

    ladder(76600469441198017145391791613091732004, 11)
```

In [8]:
u = 76600469441198017145391791613091732004
curve.mul(u, 11)

0

---
```
What the heck? What's going on here?

Let's do a quick sanity check. Here's the curve equation again:

    v^2 = u^3 + 534*u^2 + u

Plug in u and take the square root to recover v.
```

In [9]:
def u_to_v(u, curve=curve):
    p = curve.p
    v_sq = (u**3 + curve.a * u**2 + u) % p
    return tonelli_shanks(v_sq, p)

try:
    u_to_v(u)
except NoQuadraticResidueError:
    print("Square root of", u, "mod", curve.p, "does not exist!")

Square root of 76600469441198017145391791613091732004 mod 233970423115425145524320034830162017933 does not exist!


---
```
You should detect that something is quite wrong. This u does not
represent a point on our curve! Not every u does.

This means that even though we can only submit one coordinate, we
still have a little bit of leeway to find invalid
points. Specifically, an input u such that u^3 + 534*u^2 + u is not a
quadratic residue can never represent a point on our curve. So where
the heck are we?

The other curve we're on is a sister curve called a "quadratic twist",
or simply "the twist". There is actually a whole family of quadratic
twists to our curve, but they're all isomorphic to each
other. Remember that that means they have the same number of points,
the same subgroups, etc. So it doesn't really matter which particular
twist we use; in fact, we don't even need to pick one.

...

If Alice chose a curve with an insecure twist, i.e. one with a
partially smooth order, then some doors open back up for Eve. She can
choose low-order points on the twisted curve, send them to Alice, and
perform the invalid-curve attack as before.

The only caveat is that she won't be able to recover the full secret
using off-curve points, only a fraction of it. But we know how to
handle that.

So:

1. Calculate the order of the twist and find its small factors. This
   one should have a bunch under 2^24.
```

In [10]:
# The ordinary curve and its twist have 2*p + 2 points between them.
# The curve's order is known, so we can take the difference to find the twist's order:
twist_order = 2*curve.p + 2 - order

print("Twist's order:", twist_order)
print("Factoring...")
%time factors = [p for p in primegen(up_to=2**24) if twist_order % p == 0 and (twist_order // p) % p != 0]
print("Small, non-repeated factors:", factors)

Twist's order: 233970423115425145549737651362517029924
Factoring...
CPU times: user 25.6 s, sys: 363 ms, total: 26 s
Wall time: 26 s
Small, non-repeated factors: [11, 107, 197, 1621, 105143, 405373, 2323367]


----
```
2. Find points with those orders. This is simple:

   a. Choose a random u mod p and verify that u^3 + A*u^2 + u is a
      nonsquare in GF(p).

   b. Call the order of the twist n. To find an element of order q,
      calculate ladder(u, n/q).
```

In [11]:
def get_twist_point(factor):
    p = curve.p
    ladder = curve.mul

    while True:
        u = randrange(0, p)
        expr = (pow(u, 3, curve.p) + curve.a*pow(u, 2, p) + u) % curve.p
        try:
            tonelli_shanks(expr, p)  # this try clause checks to make sure u isn't on the curve
        except NoQuadraticResidueError: pass
        else: continue
        elem = ladder(u, twist_order // factor)
        if elem != 0:
            break

    assert twist_order % factor == 0
    assert ladder(elem, factor) == 0

    return elem

twist_points = {fac: get_twist_point(fac) for fac in factors}
pprint(twist_points)

print("\nHere's our order-11 twist subgroup:")
for i in range(12):
    print(curve.mul(twist_points[11], i))
print("...")

# note how the subgroup's i'th and 11-i'th elements are all equal
# this is the cause of the 'combinatorial explosion' alluded to at the end of 60.txt

{11: 105888069003703096891937904030103459645,
 107: 84353781803161890722524036605451541851,
 197: 219322255009266168993561112356879426292,
 1621: 167820136414351786653343816424225496635,
 105143: 74265784656785480718420679996162657942,
 405373: 94398023913003233927873516070605131453,
 2323367: 10247228768332257365540055800591913776}

Here's our order-11 twist subgroup:
0
105888069003703096891937904030103459645
1430388126279164727092494211327512206
76600469441198017145391791613091732004
4612483201341222105440076661179035958
173527332646559565669040569905840307859
173527332646559565669040569905840307859
4612483201341222105440076661179035958
76600469441198017145391791613091732004
1430388126279164727092494211327512206
105888069003703096891937904030103459645
0
...


---
```
3. Send these points to Alice to recover portions of her secret.
```

In [12]:
# implementation of Alice here is modeled on challenge 58's Bob
# cf that block's comments

def alice_coro(message):
    p, mul = curve.p, curve.mul
    priv = randrange(0, order)
    pub = mul(4, priv)

    h = (yield pub)
    while True:
        secret = mul(h, priv)
        K = do_sha256(int_to_bytes(secret))
        t = hmac(K, message)
        h = (yield (message, t))

alice = alice_coro(b"no alarms and no surprises")
alice_pubkey = next(alice)
print("Alice initialized. Pubkey:", alice_pubkey)

Alice initialized. Pubkey: 37262871663931412184142593413755847531


In [13]:
def get_tag(message, guess):
    K = do_sha256(int_to_bytes(guess))
    tag = hmac(K, message)
    return tag


def recover_coefficient(g, order, message, t):
    ladder = curve.mul
    for i in range(order):
        if get_tag(message, ladder(g, i)) == t:
            return (i, order-i)
    print("coefficient not found (?!)")
    raise Exception("this should never happen")

In [14]:
coeffs = {}

print("This may take a few minutes.")
for small_order, small_pt in twist_points.items():
    print("\nRecovering possible residues mod", small_order, "...", flush=True)
    message, t = alice.send(small_pt)
    %time i1, i2 = recover_coefficient(small_pt, small_order, message, t)
    coeffs[small_order] = i1, i2
    print(i1, i2)

print("Done!\n")
pprint(coeffs)

This may take a few minutes.

Recovering possible residues mod 11 ...
CPU times: user 3.07 ms, sys: 0 ns, total: 3.07 ms
Wall time: 3.07 ms
5 6

Recovering possible residues mod 107 ...
CPU times: user 16.6 ms, sys: 0 ns, total: 16.6 ms
Wall time: 16.7 ms
31 76

Recovering possible residues mod 197 ...
CPU times: user 19.9 ms, sys: 0 ns, total: 19.9 ms
Wall time: 19.9 ms
37 160

Recovering possible residues mod 1621 ...
CPU times: user 139 ms, sys: 1.59 ms, total: 141 ms
Wall time: 147 ms
165 1456

Recovering possible residues mod 105143 ...
CPU times: user 3.74 s, sys: 0 ns, total: 3.74 s
Wall time: 3.76 s
6600 98543

Recovering possible residues mod 405373 ...
CPU times: user 1min 14s, sys: 0 ns, total: 1min 14s
Wall time: 1min 14s
131029 274344

Recovering possible residues mod 2323367 ...
CPU times: user 2min 28s, sys: 0 ns, total: 2min 28s
Wall time: 2min 28s
259428 2063939
Done!

{11: (5, 6),
 107: (31, 76),
 197: (37, 160),
 1621: (165, 1456),
 105143: (6600, 98543),
 405373: (1

In [15]:
# Let's make like a tree and generate even more residues :)

def get_extra_coeffs():
    extra_coeffs = {}
    p1 = factors[0]
    for p2 in factors[1:]:
        #print(f"Recovering possible residues mod {p1}*{p2}")
        o = p1*p2
        while True:
            pt = get_twist_point(o)
            if curve.mul(pt, p1) != 0 and curve.mul(pt, p2) != 0:
                break

        message, tag = alice.send(pt)

        candidates = [mini_crt(r1, p1, r2, p2)[0] for r2 in coeffs[p2] for r1 in coeffs[p1]]

        extra_coeffs[p1, p2] = tuple(res for res in candidates
                                     if get_tag(message, curve.mul(pt, res)) == tag)
        assert extra_coeffs[p1, p2]

    print("Done!\n")
    return extra_coeffs

extra_coeffs = get_extra_coeffs()

print("Keys: Pairs of moduli (m1, m2)")
print("Values: pairs of potential residues mod m1*m2")
print()
pprint(extra_coeffs)

Done!

Keys: Pairs of moduli (m1, m2)
Values: pairs of potential residues mod m1*m2

{(11, 107): (138, 1039),
 (11, 197): (1810, 357),
 (11, 1621): (11512, 6319),
 (11, 105143): (1058030, 98543),
 (11, 405373): (3374013, 1085090),
 (11, 2323367): (2582795, 22974242)}


In [16]:
residue_pairs = []

for t1, t2 in extra_coeffs.items():
    p1, p2 = t1
    r1, r2 = t2
    pairs = (r1%p1, r1%p2), (r2%p1, r2%p2)
    residue_pairs.append(sorted(pairs))

#pprint(residue_pairs)

res_seq_1 = (coeffs[11][0],) + tuple(t[0][1] for t in residue_pairs)
res_seq_2 = (coeffs[11][1],) + tuple(t[1][1] for t in residue_pairs)

print("2 possible combinations of residues (down from 128):")
print(res_seq_1)
print(res_seq_2)

2 possible combinations of residues (down from 128):
(5, 76, 160, 1456, 98543, 131029, 2063939)
(6, 31, 37, 165, 6600, 274344, 259428)


---
```
4. When you've exhausted all the small subgroups in the twist, recover
   the remainder of Alice's secret with the kangaroo attack.
```

In [17]:
def ec_pollard(curve, g_coef, y, a, b, k):  # takes g_coef s.t. g = g_coef*G
    add_pts, mul, get_pt = curve.add_pts, curve.mul, curve.get_pt

    pt_cache = [get_pt(g_coef << u) for u in range(k)] # this cache lets us avoid costly calls like
                                                       # get_pt() and mul() in this func's hotspots

    g = curve.get_pt(g_coef)
    N = 4 * (2**k - 1) // k

    print(f"Running Pollard's algorithm (k = {k}, N = {N})")
    print()
    print("Tame kangaroo starting at:", datetime.now())

    xT = 0
    yT = get_pt(g_coef*b)
    for i in range(N):
        u = yT[0] % k
        f_yT = 1 << u
        xT += f_yT
        yT = add_pts(yT, pt_cache[u])

    t0 = datetime.now()
    print("Wild kangaroo starting at:", t0)

    xW = 0
    yW = y

    bound = b - a + xT
    update_interval = 2**21 - 1
    i = 0

    while xW < bound:
        i += 1
        if i & update_interval == 0:
            progress = xW / bound
            if (progress > 0):
                ti = datetime.now() - t0
                est_duration = ti / progress
                print(f"{datetime.now()} | {100*progress:f}% done | Appx time remaining: ({est_duration - ti} / {est_duration})")
        u = yW[0] % k
        f_yW = 1 << u
        xW += f_yW
        yW = add_pts(yW, pt_cache[u])
        
        if yW[0] == yT[0]:
            print("The wild kangaroo found something!")
            return b + xT - xW

In [18]:
# here's how that funky transform from 58.txt adapts to the elliptic curve setting:

# for some private key x, we have
# x = residue mod modulus = n mod r = n + m*r
# y' := y - n*G
# G' := r*G
# pollard's uses the fact that y' = m*G' to find m
# then we plug m into our first eqn to find x

# implementation detail: this implementation of pollard's takes r as an argument, not G' (= r*G)

modulus = reduce(mul, factors, 1)

def get_candidate_dlogs():
    dlogs = {}

    for residues in (res_seq_1, res_seq_2):
        residue, _modulus = crt(residues, factors)
        assert _modulus == modulus  # basic consistency check
        nG = curve.get_pt(residue)
        y_primes = [curve.sub_pts((alice_pubkey, v), nG) for v in curve.get_v(alice_pubkey)]

        for i, y_prime in enumerate(y_primes, start=1):
            # it's kangaroo racing time!!

            lb, ub = 0, (order // modulus) // 2
            print(f"\na, b = {lb}, {ub}\nx = {residue} mod {modulus}\ny' = {y_prime}\n")
            print(f"ec_pollard() called at {datetime.now()})")
            dlog = ec_pollard(curve, modulus, y_prime, lb, ub, k=24)
            print("Candidate dlog:", dlog)
            dlogs[residue, y_prime] = dlog
            print()
            print("----")
            print()

    return dlogs

dlogs = get_candidate_dlogs()

print("Done!\n")


a, b = 0, 3143057028025
x = 4607907578279665460348916 mod 37220200115549684379403037
y' = (93938928258205143436814261318294704461, 192753422291055068728289664629917356570)

ec_pollard() called at 2020-09-05 03:21:29.276270)
Running Pollard's algorithm (k = 24, N = 2796202)

Tame kangaroo starting at: 2020-09-05 03:21:29.332345
Wild kangaroo starting at: 2020-09-05 03:22:27.379540
2020-09-05 03:23:10.649099 | 28.866069% done | Appx time remaining: (0:01:46.628026 / 0:02:29.897558)
2020-09-05 03:23:53.541538 | 57.637249% done | Appx time remaining: (0:01:03.328113 / 0:02:29.490086)
2020-09-05 03:24:36.370882 | 86.407173% done | Appx time remaining: (0:00:20.291795 / 0:02:29.283115)
Candidate dlog: None

----


a, b = 0, 3143057028025
x = 4607907578279665460348916 mod 37220200115549684379403037
y' = (199505767643241576032520890523667401479, 228297776414974839916385969448723863589)

ec_pollard() called at 2020-09-05 03:24:56.654797)
Running Pollard's algorithm (k = 24, N = 2796202)

Tame 

In [19]:
for params, dlog in dlogs.items():
    if dlog is None:
        continue
    if dlog == -1:
        continue

    residue, y_prime = params

    print("Candidate solution:")
    print("  residue =", residue)
    print("  y' =", y_prime)
    print("  m =", dlog)
    assert y_prime[0] == curve.mul(4, modulus*dlog)
    alice_privkey_recovered = residue + dlog*modulus
    alice_pubkey_recovered = curve.mul(4, alice_privkey_recovered)
    assert alice_pubkey == alice_pubkey_recovered
    print("It works!")
    print("Alice's private key (recovered):", alice_privkey_recovered)
    print("Alice's public key (derived):   ", alice_pubkey_recovered)
    print("Alice's public key (announced): ", alice_pubkey)
    break
else:
    print("No solution found...")
    print("Try adjusting the value of k in ec_pollard() and running the search again")

Candidate solution:
  residue = 32612292537270018919054121
  y' = (203095450983231083207524911275068289655, 122942907208972815016145005312270753166)
  m = 4420875348651
It works!
Alice's private key (recovered): 164545865162723313709843679779542307208
Alice's public key (derived):    37262871663931412184142593413755847531
Alice's public key (announced):  37262871663931412184142593413755847531
